<br><br>

## Data Cleansing with Pandas

<br><br>


## Overview

#### Data Cleansing (sometimes called Data Cleaning) is the process of correcting or removing irregular values from a dataset

 - Tasks involved:
- Exploration (Detection)
        - Examine the dataset for completeness (e.g., how many missing values)
        - Verify constraints (regular expressions, type, ranges, cross-field constraints)
- Fix Anomalies (remove and/or replace)
        - Remove observations or features containing missing values
        - Impute values using different strategies
        - Remove duplicates
        - Ensure text formatting and units are consistent
        - Detect and handle outliers
- Verify Changes
 

In addition to the above, if you are data modeling, you will likely need to perform a few additional tasks:
 - Encode data values
 - Convert continuous values to categorical
 - Cut and bin continuous values
 - Perform type conversions
 - Normalize / Standarize column values

   <br><br>
   Before using this notebook, you must install dependencies with:

   <code>pip install jupyter pandas matplotlib seaborn</code>


In [ ]:
import warnings
from datetime import date

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

warnings.filterwarnings('ignore')

<br><br>
The Titanic dataset is a good place to start because it can illustrate many of the concepts discussed above. So, we'll use this dataset for examining this topic.
<br><br>

In [ ]:
# load the Titanic dataset

<br><br>
## Task 1: Data Exploration
<br>

There are 3 steps you should *always* perform after acquiring your dataset within Pandas: 
- shape
- head()
- info()

Do these **often**, even after initial data acquisition.

<br><br>
**dataframe.info()** (above) can identify many things including rows that don't have complete values.
<br>
You can also use the following statement to help detect which columns don't have values (or have NaN values)...

In [ ]:
# display NaN counts

<br><br>
Two of the columns above have missing values (embarked and embark_town).  You can look at these rows with the following statement:

In [ ]:
# display rows containing NaNs in embark_town column

<br><br>
The following shows rows where NaNs appear in *any* column...

In [ ]:
# display rows containing NaNs in any column

<br><br>
You can represent NaNs using a heatmap from the data visualization library, called Seaborn.
We could do this on multiple columns, but the others are not yet encoded, so we'll just use the 'age' column:

In [ ]:
# Display NaNs visually using a Seaborn heatmap
# Red represents a NaN

<br><br>
## Pandas Note: *df.isna()*   vs.   *ds.isnull()*
<br>
In Pandas, these methods are identical.
<br><br>
Use these methods when necessary:<br>
    Numpy...  <code>np.isnan(arr)</code><br>
    Pandas...  <code>df.isnull()</code> or <code>df.isna()</code>       
    
There are 2 ways in Pandas to mimic how it is done in R.

<br><br>
We can also view all the values in these columns to get a better picture of what they contain...

In [ ]:
# display all values found in the embark_town column

In [ ]:
# display all values found in the sibsp column

<br><br>
(above) The **unique()** function shows all the values that can occur in a specified column.  Including NaNs.
<br><br>

<br><br>
Use **value_counts()** to tell us how many can be found of a certain value in this column.  NaNs are not shown by default.  Also, this only works on a Series (single-column)

In [ ]:
# display the value counts from the embark_town column

If you want to actually see the NaNs, use dropna=False

In [ ]:
# display the value counts including the NaN values

In [ ]:
# display the value counts for the sibsp column

<br><br>
Validating Constraints

In [ ]:
# validate that all values within the pclass column are either 1, or 2, or 3 (for their class level)

<br><br>
## Task 2: Fixing Anomalies
<br>

#### Removing Observations with Missing Values


It is possible to remove *all* observations containing NaNs in any column using:
<code>titanic.dropna(inplace=True)</code>
<br>
This might be too big of an operation and may result in an unnecessary loss of data.  We'll try to handle this in a smaller, more fine-grained way, if possible...
<br><br>


Time to handle the data in the embark_town column...we'll remove the two records where NaNs occur here...
<br><br>

In [ ]:
# drop the 2 records where NaNs occur in the embark_town column

<br><br>
Once again, a check of the data...

In [ ]:
# check the info() once again...

<br><br>
The above shows 889 samples are non-null in the embark_town column now.  We can also check for NaNs directly:

In [ ]:
# check which columns still contain NaNs

<br><br>
#### Imputing Values
We'll treat the age column next by imputing values...

In [ ]:
# get the mean value in the age column (don't count NaNs by using skipna=True)

In [ ]:
# get the median value in the age column

<br><br>
To impute values, you first decide on a strategy (mean, median, some other value).  Then use fillna():

In [ ]:
# impute values within the age column (use the median value from above)

In [ ]:
# check the NaN counts again...

<br><br>
**Scikit-learn**, a Python-based data modeling tool, provides a class that can do this also, called *SimpleImputer*.  However, this solution is not quite as easy to read...

In [ ]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy='median')
column_to_impute = titanic[['age']].to_numpy()
imputed_column = imputer.fit_transform(column_to_impute)
titanic['age'] = pd.DataFrame(imputed_column)
titanic.isnull().sum()

<br><br>
#### Removing Columns
The deck has many missing values--perhaps too many to impute anything meaningful.  So, we'll simply drop this column:

In [ ]:
# drop the deck column

In [ ]:
# check the NaN counts again

<br><br>
#### Managing Duplicate Observations
Our Titanic dataset doesn't incorporate duplicate observations very well, so we'll use another sample dataset for illustration purposes.  Consider the following dataset:

In [ ]:
ice_cream = pd.DataFrame({
    'flavor': ['Vanilla', 'Chocolate', 'Strawberry', 'Vanilla', 'Vanilla', 'Chocolate', 'Vanilla'],
    'topping': ['None', 'Sprinkles', 'Hot Fudge', 'Sprinkles', 'Sprinkles', 'Caramel', 'None'],
    'cost': [4, 4.75, 4.75, 4.75, 4.75, 4.75, 4]
})
ice_cream

<br><br>
We can use **drop_duplicates()** to remove any observations that have identical values in all of the corresponding columns...

In [ ]:
# remove duplicate records

<br><br>
If we only wish to consider duplication of data in a specified column or columns, use **subset=**.

In [ ]:
# remove duplicate flavors

<br><br>
#### Text Formatting and Pandas Accessors
Pandas provides three accessors (str, dt, cat) that allow column operations element-by-element

In [ ]:
personnel = pd.DataFrame([('Edward', 'Janitor', date(2004, 3, 3)),
                   ('Tracy', 'Teacher', date(2017, 7, 17)),
                   ('Amanda', 'Bus Driver', date(2011, 5, 21))],
                  columns=['Name', 'Position', 'Hire_date'])
personnel

In [ ]:
# uppercase the Names

In [ ]:
# Encode the Positions

In [ ]:
# Determine the month of hire

<br><br>
## Encoding Values
Now for some data encoding...
<br><br>
First, let's look at the data types for the columns...

In [ ]:
# Examine the dtypes of the columns

In [ ]:
# check the types using info()...

<br><br>
Columns that are object types are not useful when creating models.  They need to be converted or encoded.
The easiest way to encode values is through **simple value-replacement**.  

In [ ]:
# check the value_counts() of the embarked column

In [ ]:
# check the value_counts() of the 'who' column

In [ ]:
# use value replacement to replace the values in the embarked and who columns

In [ ]:
# sample the first 5 rows of the dataframe

In [ ]:
# check the info() again...

<br><br> 
Now the who and embarked columns are int dtypes.
<br><br>
For the embark_town column, **we'll use categorical encoding**.  First it needs to be converted to a category type:

In [ ]:
# examine the value_counts() of the embark_town column

In [ ]:
# convert the embark_town column to a category type

In [ ]:
# Use the codes attribute of the cat accessor to encode the embark_town column

In [ ]:
# examine the value_counts() of embark_town again...

In [ ]:
# sample the first 5 records of the dataframe again

In [ ]:
# check the info() function to see the column types again

<br><br>
The class column is a category type, but it contains strings, which can cause problems.  Let's **one-hot encode** this column using Pandas **get_dummies(df, columns)**:

In [ ]:
# use get_dummies() to one-hot encode the 'class' column.  Examine the first 5 rows of the resulting dataframe.

<br><br>
The alive column represents similar information to the survived label and the along column represents similar information to the sibsp.  The adult_male and sex columns are also covered by the "who" column.  So, **we'll drop these four columns**:

In [ ]:
# drop the above mentioned columns
# view the first 5 rows of the resulting dataframe

<br><br>
Even though we did this using Pandas get_dummies() already, as a demonstration, we'll convert the **pclass** column into a one-hot encoded solution **using Scikit-learn's LabelBinarizer**:

In [ ]:
from sklearn.preprocessing import LabelBinarizer

In [ ]:
lb = LabelBinarizer()
pclass_encoded = pd.DataFrame(lb.fit_transform(titanic.pclass), columns=['p1', 'p2', 'p3'])

In [ ]:
titanic = pd.concat([titanic, pclass_encoded], axis=1, join='inner')
titanic.drop('pclass', axis=1, inplace=True)
titanic.head()

<br><br>
#### Normalization and Standardization
For some algorithms, large values can affect results.  Often using normalization and scaling can help with results.
<br>
We'll use a StandardScaler to change those values.  Examples of StandardScaler and MinMaxScaler...

<br><br>

**Z-score (Standardization) Scaling** - transforms features so that they have a mean = 0, and a std dev. of 1.<br>
**Min-Max Scaling (Normalization)** -  Scales values down to fit into a specified range (usually 0 to 1).
<br>
To help understand each, take the following example...

In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [ ]:
data = [0, 1, 2]
data = pd.DataFrame(data)
data

In [ ]:
data_scaled = StandardScaler().fit_transform(data)
data_scaled = pd.DataFrame(data_scaled)
data_scaled

In [ ]:
data_minmax = MinMaxScaler().fit_transform(data)
data_minmax = pd.DataFrame(data_minmax)
data_minmax

<br><br>
We'll use Standardization for the fare column...

In [ ]:
titanic.fare.to_numpy().reshape(-1, 1)[:5]      # a sample of the first 5 records

In [ ]:
# Use a standardscaler to scale the values in the fare column.  View the first 5 rows of the resulting df

<br><br>
## Outlier Detection Analysis and Removal
<br><br>
Lastly, we'll perform an outlier detection analysis...

In [ ]:
sns.boxplot(data=titanic, x=titanic.age);

In [ ]:
sns.boxplot(data=titanic, x=titanic.fare);

In [ ]:
def remove_outliers_by_column(df, column, whisker=1.5):
    Q1 = df[column].quantile(0.25)  # calculate Q1
    Q3 = df[column].quantile(0.75)  # calculate Q3
    IQR = Q3 - Q1
    width = whisker * IQR
    lower_bound = Q1 - width
    upper_bound = Q3 + width
    filter = (df[column] >= lower_bound) &  (df[column] <= upper_bound)
    return df.loc[filter]

In [ ]:
titanic.shape

In [ ]:
# remove outliers from the 'fare' column
# view the resulting shape

In [ ]:
# remove outliers from the 'age' column
# view the resulting shape

In [ ]:
# with outliers removed, assign the titanic to the value returned from the last function call

In [ ]:
# sample the first five rows of our df once again...

In [ ]:
# sample the column structure (using info() again)

In [ ]:
# determine if we have any remaining NaN values

In [ ]:
# downsize the columns

In [ ]:
# check the column structure one last time

In [ ]:
# sample the dataframe one lst time